# Recursive Feature Elimination (RFE)

* Automatically finds the most important features by using cross-validation and discarding the rest.
* Create a fake dataset of 15 features, 10 of which are informative, and the rest are redundant.
* Fit a 5-fold RFECV with `Ridge` regression as an estimator.
* After training, use the `transform` method to discard the redundant features.
* Calling `.shape` shows that the estimator managed to drop all 5 unnecessary features.

In [1]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Ridge

In [2]:
def get_model_coefficients(X_train, model):
    return pd.DataFrame(
        zip(X_train.columns, abs(model.feature_importances_)),
        columns=["feature", "weight"],
        ).sort_values("weight").reset_index(drop=True)

In [3]:
# Build a synthetic dataset
X, y = make_regression(n_samples=10000, n_features=15, n_informative=10)

# Init/fit the selector
rfecv = RFECV(estimator=Ridge(), cv=5)
_ = rfecv.fit(X, y)

# Transform the feature array
print(rfecv.transform(X).shape)
rfecv.transform(X)

(10000, 10)


array([[-0.31515513,  0.4540941 , -0.06624467, ..., -0.55378483,
         0.71184009,  0.44486228],
       [ 0.6065811 ,  1.43812206, -0.05342859, ...,  0.51570444,
         0.26159076,  0.93517493],
       [ 1.22746837, -0.31445629,  0.29528331, ..., -2.01382337,
        -0.28925247,  0.61773845],
       ...,
       [ 1.24816453, -0.32682348,  0.12148773, ...,  1.4741984 ,
        -0.14154531,  0.10864823],
       [ 0.79025152,  1.19168395, -0.62832295, ..., -0.36461622,
        -0.63567108,  0.34771085],
       [-0.50133269,  0.22721426,  0.41357835, ..., -0.78072342,
         1.1629103 , -0.57966704]])

## Ansur dataset

* 6000 US Male Army Personnel.
* Records more than 100 different types of body measurements.
* Challenge is to predict the weight in pounds using as few features as possible.

In [21]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE, RFECV

In [5]:
DATA_DIR = '/workspaces/sklearn_tricks/data/'

# Load dataset
ansur = pd.read_csv(DATA_DIR + 'Ansur.csv', encoding='latin1')

### Managing the data

Dataset has 9 non-numerical columns:

| Feature               | # unique values  |
|-----------------------|-----|
| Gender                |   1 |
| Date                  | 185 |
| Installation          |  11 |
| Component             |   3 |
| Branch                |   3 |
| PrimaryMOS            | 229 |
| SubjectsBirthLocation | 126 |
| Ethnicity             | 159 |
| WritingPreference     |   3 |

### Strategy #1 - ignore all non-numerical values

* We lose all 9 features.
* We still have 98 features for the model to learn from.

In [6]:
# Find non-numeric columns
numeric_cols = ansur.select_dtypes(include=['number']).columns
ansur_numeric = ansur.select_dtypes(include=['number'])

# Drop numerical columns
ansur_non_numeric = ansur.drop(columns=numeric_cols)
print(ansur_non_numeric.nunique())

Gender                     1
Date                     185
Installation              11
Component                  3
Branch                     3
PrimaryMOS               229
SubjectsBirthLocation    126
Ethnicity                159
WritingPreference          3
dtype: int64


### Set target variable (y) and feature vector (X) 

In [7]:
# Get target (Weightlbs) and remaining 107 features
y = ansur_numeric['Weightlbs']
X = ansur_numeric.drop('Weightlbs', axis=1)

### Scale the data

In [8]:
# Train/test set generation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Scale train and test sets with StandardScaler
X_train_std = StandardScaler().fit_transform(X_train)
X_test_std = StandardScaler().fit_transform(X_test)

# Fix the dimensions of the target array
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

### Fit a Random Forest regression model

* Base model
* Achieves R-squared score of 0.957

In [10]:
# Init, fit, test Lasso Regressor
forest = RandomForestRegressor()
_ = forest.fit(X_train_std, y_train.ravel())
forest.score(X_test_std, y_test)

0.95893533120974

### Feature weights

* Model has feature coefficients (weights)
* Use the model .feature_importances_ method to display each feature's weight
* Weights near zero contribute little to the model
* CAUTION:
  * Removing a single feature forces other coefficients to change
  * We cannot just remove multiple features
  * Instead, need to remove one at a time
  * Sklearn provides the RFE class to automate this process

In [11]:
get_model_coefficients(X_train, model=forest)

,feature,weight
0,DODRace,0.000061
1,cervicaleheight,0.000159
2,SubjectNumericRace,0.000163
3,Heightin,0.000176
4,suprasternaleheight,0.000181
...,...,...
93,forearmcircumferenceflexed,0.002415
94,shouldercircumference,0.002442
95,bimalleolarbreadth,0.002496
96,bideltoidbreadth,0.004124


### Sklearn Recursive Feature Elimination class

* Removes features one at a time based on the weights given by a model in each iteration

In [12]:
# Init the transformer
rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=10)

# Fit to the training data
_ = rfe.fit(X_train_std, y_train.ravel())

In [15]:
# NumPy array of just 10 most important features
ansur_rfe_10 = rfe.transform(X_train_std)

### Fit a Random Forest regression model on 10 features

* Base model
* Achieves R-squared score of 0.956
* With 98 features the R-squared score was 0.957

In [17]:
# Init, fit, score
forest = RandomForestRegressor()
_ = forest.fit(ansur_rfe_10, y_train.ravel())
forest.score(rfe.transform(X_test_std), y_test)

0.9557863523459048

### RFE performance considerations

* RFE trains a given model on the FULL DATASET every time it drops a feature
* Computation time is long for large datasets with many features
* Use the step parameter to drop N features instead of 1 in each iteration
* In this example, we use step = 10
* The RFE was calculated in 1 minute 15 seconds versus 11 minutes 27 seconds when only 1 feature was dropped at each iteration

In [19]:
# Init the transformer
rfe = RFE(estimator=RandomForestRegressor(),
          n_features_to_select=10,
          step=10)

_ = rfe.fit(X_train_std, y_train.ravel())

print(X_train.columns[rfe.support_])

Index(['bicepscircumferenceflexed', 'bideltoidbreadth', 'bimalleolarbreadth',
       'bizygomaticbreadth', 'forearmforearmbreadth', 'handcircumference',
       'neckcircumferencebase', 'shouldercircumference', 'weightkg',
       'wristheight'],
      dtype='object')


### Automatically choosing number of features

* We arbitrarily chose 10 features
* We can use RFE with cross-validation to find the optimal number of features to keep
* This is the RFECV class
* In this example, we choose the estimator to be a simple linear regression instead of random forest which will be computationally expensive
* We can use linear regression since our hypothesis states that there is a linear correlation between body measurements
* RFECV suggests we keep only 5 features
* Our R-squared score is 0.963 with only 5 features

In [24]:
# Init, fit
rfecv = RFECV(
    estimator=LinearRegression(),
    min_features_to_select=5,
    step=5,
    n_jobs=-1,
    scoring="r2",
    cv=5,
)

_ = rfecv.fit(X_train_std, y_train)

print(X_train.columns[rfecv.support_])

Index(['acromialheight', 'bideltoidbreadth', 'shouldercircumference',
       'stature', 'weightkg'],
      dtype='object')


In [25]:
lr = LinearRegression()
_ = lr.fit(X_train_std, y_train)

print(f"Training R-squared: {lr.score(X_train_std, y_train)}")
print(f"Testing R-squared: {lr.score(X_test_std, y_test)}")

Training R-squared: 0.9387440321037271
Testing R-squared: 0.962870350802704
